In [1]:
import os
%pwd
os.chdir('../')
%pwd

'C:\\Users\\Franklin NDJATHE\\Documents\\projects\\End-to-End-ML-Project-with-MLFlow'

In [4]:
from dataclasses import  dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [6]:
from typing import ClassVar
from box import ConfigBox

@dataclass
class ConfigurationManager:
    config_filepath: Path = CONFIG_FILE_PATH
    params_filepath: Path = PARAMS_FILE_PATH
    schema_filepath: Path = SCHEMA_FILE_PATH
    
    config: ClassVar[ConfigBox] = read_yaml(config_filepath)
    params: ClassVar[ConfigBox] = read_yaml(params_filepath)
    schema: ClassVar[ConfigBox] = read_yaml(schema_filepath)
    
    def __post_init__(self):
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        
        return model_trainer_config

[2023-07-29 20:19:43,571: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-29 20:19:43,573: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-29 20:19:43,576: INFO: common: yaml file: schema.yaml loaded successfully]


In [7]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

[2023-07-29 20:20:29,747: INFO: utils: NumExpr defaulting to 8 threads.]


In [8]:
@dataclass
class ModelTrainer:
    config: ModelTrainerConfig
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-07-29 20:27:52,512: INFO: common: created directory at: artifacts]
[2023-07-29 20:27:52,513: INFO: common: created directory at: artifacts/model_trainer]
